In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing the dataset

In [ ]:
dataset_train = pd.read_csv("/kaggle/input/DontGetKicked/training.csv")
dataset_test = pd.read_csv("/kaggle/input/DontGetKicked/test.csv")

## Data Preprocessing

In [ ]:
dataset_test.info()

In [ ]:
dataset_train.isnull().sum()

In [ ]:
dataset_test.isnull().sum()

### Replace Unwanted Columns

In [ ]:
dataset_train.drop(['Model', 'SubModel'], axis=1, inplace=True)
dataset_test.drop(['Model', 'SubModel'], axis=1, inplace=True)

In [ ]:
dataset_train['Color'].fillna(value='COLOR_UNKNOWN', inplace=True)
dataset_test['Color'].fillna(value='COLOR_UNKNOWN', inplace=True)
dataset_train['Transmission'].replace(to_replace='Manual', value='MANUAL', inplace=True)
dataset_test['Transmission'].replace(to_replace='Manual', value='MANUAL', inplace=True)
dataset_train['Transmission'].fillna(value='TRANS_UNKNOWN', inplace=True)
dataset_test['Transmission'].fillna(value='TRANS_UNKNOWN', inplace=True)
dataset_train.drop(['WheelTypeID'], axis=1, inplace=True)
dataset_test.drop(['WheelTypeID'], axis=1, inplace=True)
dataset_train['Nationality'].fillna(value='NATIONALITY_UNKNOWN', inplace=True)
dataset_test['Nationality'].fillna(value='NATIONALITY_UNKNOWN', inplace=True)
dataset_train['Size'].fillna(value='SIZE_UNKNOWN', inplace=True)
dataset_test['Size'].fillna(value='SIZE_UNKNOWN', inplace=True)
dataset_train['TopThreeAmericanName'].fillna(value='TOP_UNKNOWN', inplace=True)
dataset_test['TopThreeAmericanName'].fillna(value='TOP_UNKNOWN', inplace=True)
dataset_train['PRIMEUNIT'].fillna(value='PRIME_UNKNOWN', inplace=True)
dataset_test['PRIMEUNIT'].fillna(value='PRIME_UNKNOWN', inplace=True)
dataset_train['AUCGUART'].fillna(value='AUC_UNKNOWN', inplace=True)
dataset_test['AUCGUART'].fillna(value='AUC_UNKNOWN', inplace=True)
dataset_train.drop(['MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice',
             'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice', 
             'MMRCurrentRetailCleanPrice'], inplace=True, axis=1)

dataset_test.drop(['MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice',
             'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice', 
             'MMRCurrentRetailCleanPrice'], inplace=True, axis=1)
dataset_test.drop(['Trim'], axis=1, inplace=True)
dataset_train.drop(['Trim'], axis=1, inplace=True)
dataset_train['WheelType'].fillna(value='WHEELTYPE_UNKNOWN', inplace=True)
dataset_test['WheelType'].fillna(value='WHEELTYPE_UNKNOWN', inplace=True)
dataset_test.drop(['PurchDate'], axis=1, inplace=True)
dataset_train.drop(['PurchDate'], axis=1, inplace=True)

In [ ]:
dataset_test

In [ ]:
dataset_train.shape

In [ ]:
dataset_test.shape

## Find out list of categorical columns and apply one hot encoding

In [ ]:
categorical_columns = dataset_train.columns[dataset_train.dtypes == 'object']
non_categorical_columns = dataset_train.drop(['RefId', 'IsBadBuy'], axis=1).columns[dataset_train.drop(['RefId', 'IsBadBuy'], axis=1).dtypes != 'object']

In [ ]:
dataset_test.dtypes

In [ ]:
categorical_columns

In [ ]:
non_categorical_columns

## Apply Feature Scaling in the non-categorical-columns

In [ ]:
for col in non_categorical_columns:
    maximum_train = np.max(dataset_train[col])
    dataset_train[col] = dataset_train[col] / maximum_train
    maximum_test = np.max(dataset_test[col])
    dataset_test[col] = dataset_test[col] / maximum_test

In [ ]:
dataset_train[non_categorical_columns]

In [ ]:
dataset_test[non_categorical_columns]

In [ ]:
new_ds = dataset_train

## categorical Encoding For test set and training set

In [ ]:
for i in new_ds.index:
    for col_name in categorical_columns:
        new_ds.at[i, col_name] = new_ds.at[i, col_name] + "_" + col_name
        
for i in dataset_test.index:
    for col_name in categorical_columns:
        dataset_test.at[i, col_name] = dataset_test.at[i, col_name] + "_" + col_name

In [ ]:
new_ds

In [ ]:
dataset_test

In [ ]:
cat_train_df = pd.get_dummies(new_ds[categorical_columns])
non_cat_train_df = new_ds.select_dtypes(exclude=['object']).drop('IsBadBuy', axis=1)
final_train_dataset = pd.concat([cat_train_df, non_cat_train_df], axis=1)

In [ ]:
final_train_dataset.shape

In [ ]:
cat_test_df = pd.get_dummies(dataset_test[categorical_columns])
non_cat_test_df = dataset_test.select_dtypes(exclude=['object'])
final_test_dataset = pd.concat([cat_test_df, non_cat_test_df], axis=1)

In [ ]:
final_test_dataset.shape

## Adding the train data columns that is not in test data

In [ ]:
for cols in final_train_dataset.columns:
    if cols not in final_test_dataset.columns:
        final_test_dataset[cols] = np.zeros(len(final_test_dataset))

In [ ]:
for cols in final_test_dataset.columns:
    if cols not in final_train_dataset.columns:
        final_train_dataset[cols] = np.zeros(len(final_train_dataset))

In [ ]:
final_train_dataset.columns

In [ ]:
final_test_dataset.columns == final_train_dataset.columns

In [ ]:
final_test_dataset = final_test_dataset[final_train_dataset.columns]

In [ ]:
final_train_dataset.columns == final_test_dataset.columns

In [ ]:
final_test_dataset.columns

In [ ]:
## Making Training set and test set

In [ ]:
X = final_train_dataset.drop('RefId', axis=1).iloc[:,:].values
y = dataset_train.iloc[:,1].values
X_submission = final_test_dataset.drop('RefId', axis=1).iloc[:,:].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_submission.shape

## Splitting the dataset into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Applying KNN Model On X_train, y_train

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 10, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

## Making predictions in submission set

In [ ]:
y_submission = classifier.predict(X_submission)

In [ ]:
y_submission

In [ ]:
ref_ids = dataset_test.iloc[:,0].values

In [ ]:
submission_dataset = pd.DataFrame({'RefId':ref_ids, 'IsBadBuy':y_submission})

In [ ]:
submission_dataset.set_index('RefId', inplace=True)

In [ ]:
submission_dataset

In [ ]:
submission_dataset.to_csv('Submission.csv')